# Manipulate a function
Tutorial by Jonas Wilfert, Tobias Thummerer

## License
Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert

Licensed under the MIT license. See [LICENSE](https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multithreading**, check `distributed.ipynb` for multiprocessing.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/parallel.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install Folds via                 | add Folds                 |                                                |
| 6.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section

To run the example, the previously installed packages must be included. 

In [1]:
# imports
using FMI
using FMIZoo
using Folds
using BenchmarkTools

Checking the amount of threads:

In [2]:
Threads.nthreads()

16

### Simulation setup

Next, the start time and end time of the simulation are set. Here we also decide the size of the batch.

In [9]:
t_start = 0.0
t_step = 0.1
t_stop = 10.0
tspan = (t_start, t_stop)
tData = collect(t_start:t_step:t_stop)

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))


Single Threaded


16-element Vector{Vector{Float64}}:
 [0.08909889898036016, 0.11535912388379621]
 [0.9544910405291591, 0.4768235227760763]
 [0.4497760704115781, 0.3030216445877333]
 [0.509253718461119, 0.7206748391205103]
 [0.29343085626184584, 0.4317434043193026]
 [0.594647813518674, 0.20283512841395845]
 [0.06614984332278828, 0.8535071296214761]
 [0.0636401184871418, 0.6499795750108074]
 [0.5636210000570769, 0.20490909458217477]
 [0.5398969224823059, 0.9034419896341969]
 [0.3739975525521997, 0.45863961283510934]
 [0.4763483048959588, 0.7081906861097439]
 [0.662582928590863, 0.38170479516880784]
 [0.4698732095630027, 0.9344791524452782]
 [0.9752843499035083, 0.6865401662307917]
 [0.2365113429440906, 0.5216684087421567]

We need to instantiate one FMU for each parallel execution, as they cannot share state.

In [ ]:
realFMU = fmiLoad("SpringPendulum1D", "Dymola", "2022x")
fmiInstantiate!(realFMU)


realFMUBatch = [fmiLoad("SpringPendulum1D", "Dymola", "2022x") for _ in 1:batchSize]
fmiInstantiate!.(realFMUBatch)

We define a helper function to calculate the FMU and combine it into an Matrix.

In [6]:
function runCalcFormatted(fmu::FMU2, x0::Vector{Float64}, recordValues::Vector{String}=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, t_start, t_stop; recordValues=recordValues, saveat=tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

runCalcFormatted (generic function with 2 methods)

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [7]:
@benchmark data = runCalcFormatted(realFMU, rand(2))

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  317.864 ms … 331.613 ms  ┊ GC (min … max): 2.09% … 4.48%
 Time  (median):     324.870 ms               ┊ GC (median):    3.93%
 Time  (mean ± σ):   324.874 ms ±   4.285 ms  ┊ GC (mean ± σ):  3.55% ± 0.91%

  ▁▁    ▁           ▁  ▁ ▁▁▁           ▁▁  ▁   █     ▁   ▁    ▁  
  ██▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁█▁███▁▁▁▁▁▁▁▁▁▁▁██▁▁█▁▁▁█▁▁▁▁▁█▁▁▁█▁▁▁▁█ ▁
  318 ms           Histogram: frequency by time          332 ms <

 Memory estimate: 119.32 MiB, allocs estimate: 3002341.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [15]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(realFMU, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 5.167 s (3.77% GC) to evaluate,
 with a memory estimate of 1.86 GiB, over 48037444 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple threads, the library `Folds` can be used.

In [14]:
println("Multi Threaded")
@benchmark Folds.collect(runCalcFormatted(fmu, i) for (fmu, i) in zip(realFMUBatch, input_values))

Multi Threaded


BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.182 s …    1.844 s  ┊ GC (min … max): 36.00% … 56.80%
 Time  (median):     1.732 s               ┊ GC (median):    55.43%
 Time  (mean ± σ):   1.623 s ± 298.264 ms  ┊ GC (mean ± σ):  52.28% ± 10.04%

  █                                              ██        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁█ ▁
  1.18 s         Histogram: frequency by time         1.84 s <

 Memory estimate: 1.86 GiB, allocs estimate: 48037644.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [17]:
fmiUnload(realFMU)
fmiUnload.(realFMUBatch)

16-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

### Summary

In this tutorial it is shown how multi threading with `Folds.jl` can be used to improve the performance for calculating a Batch of FMUs.